<a href="https://colab.research.google.com/github/acreatorrr/Fashion-Review-Model/blob/main/Fashion_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

paramaggarwal_fashion_product_images_dataset_path = kagglehub.dataset_download('paramaggarwal/fashion-product-images-dataset')
praveenavala_not_seen_path = kagglehub.dataset_download('praveenavala/not-seen')

print('Data source import complete.')


In [ ]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import os
import matplotlib.pyplot as plt

In [ ]:
# Load the Dataset

styles_df = pd.read_csv('/kaggle/input/fashion-product-images-dataset/fashion-dataset/styles.csv', on_bad_lines='skip')
print(styles_df.head())

In [ ]:
# Update the image path
styles_df['image_paths'] = styles_df['id'].apply(lambda x: f'/kaggle/input/fashion-product-images-dataset/fashion-dataset/images/{x}.jpg')

In [ ]:
# Display the first few rows of the updated DataFrame
print(styles_df.head())

In [ ]:
import seaborn as sns
# Plot distribution of articleType
plt.figure(figsize=(8, 16))
sns.countplot(y='articleType', data=styles_df, order=styles_df['articleType'].value_counts().index)
plt.title('Distribution of Product Types')
plt.show()

In [ ]:
# Plot
plt.figure(figsize=(12, 6))
sns.countplot(y='baseColour', data=styles_df, order=styles_df['baseColour'].value_counts().index)
plt.title('Distribution of Product Colors')
plt.show()

In [ ]:
styles_df=styles_df[0:20000]
styles_df.shape

In [ ]:
#  Missing Values
styles_df.dropna(subset=['articleType'], inplace=True)
print(styles_df.isnull().sum())

In [ ]:
# Encode labels
article_encoder = LabelEncoder()
styles_df['articleType_encoded'] = article_encoder.fit_transform(styles_df['articleType'])

In [ ]:
def image_exists(image_paths):
    return os.path.exists(image_paths)

styles_df = styles_df[styles_df['image_paths'].apply(image_exists)]
print(f"Number of rows after filtering: {len(styles_df)}")

In [ ]:
article_encoder = LabelEncoder()
color_encoder = LabelEncoder()
season_encoder = LabelEncoder()
gender_encoder = LabelEncoder()

styles_df['articleType_encoded'] = article_encoder.fit_transform(styles_df['articleType'])
styles_df['baseColour_encoded'] = color_encoder.fit_transform(styles_df['baseColour'])
styles_df['season_encoded'] = season_encoder.fit_transform(styles_df['season'])
styles_df['gender_encoded'] = gender_encoder.fit_transform(styles_df['gender'])

In [ ]:
def preprocess_image(image_paths, target_size=(64, 64)):
    image = tf.keras.preprocessing.image.load_img(image_paths, target_size=target_size)
    image = tf.keras.preprocessing.image.img_to_array(image) / 255.0
    return image

In [ ]:
def prepare_dataset(df):
    images = np.array([preprocess_image(path) for path in df['image_paths']])
    article_labels = to_categorical(df['articleType_encoded'], num_classes=len(article_encoder.classes_))
    color_labels = to_categorical(df['baseColour_encoded'], num_classes=len(color_encoder.classes_))
    season_labels = to_categorical(df['season_encoded'], num_classes=len(season_encoder.classes_))
    gender_labels = to_categorical(df['gender_encoded'], num_classes=len(gender_encoder.classes_))
    return images, [article_labels, color_labels, season_labels, gender_labels]

In [ ]:
train_df, test_df = train_test_split(styles_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

In [ ]:
train_images, train_labels = prepare_dataset(train_df)
val_images, val_labels = prepare_dataset(val_df)
test_images, test_labels = prepare_dataset(test_df)

In [ ]:
input_shape = (64, 64, 3)

# Base model (CNN)
inputs = layers.Input(shape=input_shape)
x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)


article_output = layers.Dense(len(article_encoder.classes_), activation='softmax', name='article_output')(x)
color_output = layers.Dense(len(color_encoder.classes_), activation='softmax', name='color_output')(x)
season_output = layers.Dense(len(season_encoder.classes_), activation='softmax', name='season_output')(x)
gender_output = layers.Dense(len(gender_encoder.classes_), activation='softmax', name='gender_output')(x)



In [ ]:
model = models.Model(inputs, [article_output, color_output, season_output, gender_output])

model.compile(optimizer='adam',
              loss={'article_output': 'categorical_crossentropy',
                    'color_output': 'categorical_crossentropy',
                    'season_output': 'categorical_crossentropy',
                    'gender_output': 'categorical_crossentropy'},
              metrics={'article_output': 'accuracy',
                       'color_output': 'accuracy',
                       'season_output': 'accuracy',
                       'gender_output': 'accuracy'})


model.summary()

In [ ]:
history = model.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    epochs=10, batch_size=32)

In [ ]:
# Plot
plt.plot(history.history['article_output_accuracy'], label='Article Training Accuracy')
plt.plot(history.history['val_article_output_accuracy'], label='Article Validation Accuracy')
plt.plot(history.history['color_output_accuracy'], label='Color Training Accuracy')
plt.plot(history.history['val_color_output_accuracy'], label='Color Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Evaluate on the test set
test_loss, test_article_loss, test_color_loss, test_season_loss, test_gender_loss, \
test_article_acc, test_color_acc, test_season_acc, test_gender_acc = model.evaluate(test_images, test_labels)

print(f"Test Article Accuracy: {test_article_acc}")
print(f"Test Color Accuracy: {test_color_acc}")
print(f"Test Season Accuracy: {test_season_acc}")
print(f"Test Gender Accuracy: {test_gender_acc}")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


model = load_model('fashion_model_cnn.h5')

def predict_fashion(image_path):

    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')
    plt.title("Input Image")
    plt.show()

    img = image.load_img(image_path, target_size=(64, 64))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    predictions = model.predict(img_array)
    article_pred = np.argmax(predictions[0], axis=1)
    color_pred = np.argmax(predictions[1], axis=1)
    season_pred = np.argmax(predictions[2], axis=1)
    gender_pred = np.argmax(predictions[3], axis=1)

    article_type = article_encoder.inverse_transform(article_pred)[0]
    color = color_encoder.inverse_transform(color_pred)[0]
    season = season_encoder.inverse_transform(season_pred)[0]
    gender = gender_encoder.inverse_transform(gender_pred)[0]

    return article_type, color, season, gender

image_path = '/kaggle/input/not-seen/black_st.jpg'
article_type, color, season, gender = predict_fashion(image_path)
print(f"Article Type: {article_type}")
print(f"Color: {color}")
print(f"Season: {season}")
print(f"Gender: {gender}")

In [ ]:
image_path = '/kaggle/input/not-seen/blu_t_womwn.jpg'
article_type, color, season, gender = predict_fashion(image_path)
print(f"Article Type: {article_type}")
print(f"Color: {color}")
print(f"Season: {season}")
print(f"Gender: {gender}")

In [ ]:
image_path = '/kaggle/input/not-seen/blue_jn.jpg'
article_type, color, season, gender = predict_fashion(image_path)
print(f"Article Type: {article_type}")
print(f"Color: {color}")
print(f"Season: {season}")
print(f"Gender: {gender}")

In [ ]:
image_path = '/kaggle/input/not-seen/socks.jpg'
article_type, color, season, gender = predict_fashion(image_path)
print(f"Article Type: {article_type}")
print(f"Color: {color}")
print(f"Season: {season}")
print(f"Gender: {gender}")